In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
cd /content/gdrive/MyDrive/SemEval2023/SemEval2022-Task10/private_space

/content/gdrive/.shortcut-targets-by-id/1lC-ZKLaCDQyfLcof2Ak7FDa6IvTt318A/SemEval2023/SemEval2022-Task10/private_space


In [14]:
# !git pull origin main

In [15]:
import pandas as pd
# unsupervised_data_gab = pd.read_csv("Data/starting_ki/gab_1M_unlabelled.csv")
# unsupervised_data_reddit = pd.read_csv("Data/starting_ki/reddit_1M_unlabelled.csv")

In [18]:
from sklearn.model_selection import train_test_split

# target_column = "label_category"
# data = pd.read_csv("Data/starting_ki/train_all_tasks.csv")

In [19]:
ls ../GeneratedTexts

'Cleaning Generated Data.ipynb'   TaskC/
 TaskB/                           task_c_fill_in_blank.csv
 task_b_fill_in_blank.csv         task_c_generated_text.csv
 task_b_generated_text.csv


In [47]:
target_column = "label_category"

data = pd.read_csv("../GeneratedTexts/task_b_generated_text.csv")
data[target_column] = data["label"]

In [48]:
data = data[data[target_column]!="none"]

In [50]:
if target_column == "label_category":
  label_values = [
      '1. threats, plans to harm and incitement',
      '2. derogation',
      '3. animosity',
      '4. prejudiced discussions',
  ]

#   label_map = {
#       0: '1. threats, plans to harm and incitement',
#       1: '2. derogation', 
#       2: '3. animosity', 
#       3: '4. prejudiced discussions',
#       '1. threats, plans to harm and incitement':0,
#       '2. derogation':1,
#       '3. animosity':2,
#       '4. prejudiced discussions':3,
#   }
elif target_column == "label_vector":
  label_values = [
      '1.1 threats of harm',
      '1.2 incitement and encouragement of harm',
      '2.1 descriptive attacks',
      '2.2 aggressive and emotive attacks',
      '2.3 dehumanising attacks & overt sexual objectification',
      '3.1 casual use of gendered slurs, profanities, and insults',
      '3.2 immutable gender differences and gender stereotypes',
      '3.3 backhanded gendered compliments',
      '3.4 condescending explanations or unwelcome advice',
      '4.1 supporting mistreatment of individual women',
      '4.2 supporting systemic discrimination against women as a group',
  ]

#   label_map = {
#       0: '1.1 threats of harm',
#       1: '1.2 incitement and encouragement of harm',
#       2: '2.1 descriptive attacks',
#       3: '2.2 aggressive and emotive attacks',
#       4: '2.3 dehumanising attacks & overt sexual objectification',
#       5: '3.1 casual use of gendered slurs, profanities, and insults',
#       6: '3.2 immutable gender differences and gender stereotypes',
#       7: '3.3 backhanded gendered compliments',
#       8: '3.4 condescending explanations or unwelcome advice',
#       9: '4.1 supporting mistreatment of individual women',
#       10: '4.2 supporting systemic discrimination against women as a group',
#       '1.1 threats of harm': 0,
#       '1.2 incitement and encouragement of harm': 1,
#       '2.1 descriptive attacks': 2,
#       '2.2 aggressive and emotive attacks': 3,
#       '2.3 dehumanising attacks & overt sexual objectification': 4,
#       '3.1 casual use of gendered slurs, profanities, and insults': 5,
#       '3.2 immutable gender differences and gender stereotypes': 6,
#       '3.3 backhanded gendered compliments': 7,
#       '3.4 condescending explanations or unwelcome advice': 8,
#       '4.1 supporting mistreatment of individual women': 9,
#       '4.2 supporting systemic discrimination against women as a group': 10,
#   }
# else:
#   raise Exception("Unknown label column")

## Filter out some data

In [23]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.5 MB/s eta 0:00:00


In [24]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [25]:
import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig

out_dir = f'../Models/1a_fine-tuned-bert'
model = BertForSequenceClassification.from_pretrained(out_dir)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [26]:
import transformers
import torch
import numpy as np
import scipy as sp

In [51]:
texts = data["text"].values

In [52]:
from tqdm import tqdm
text_ids = []
for text in tqdm(texts, total=len(texts)):
  ids = tokenizer.encode(text, max_length=100, padding='max_length', truncation=True)
  text_ids.append(ids)

text_ids_lengths = [len(text_ids[i]) for i in range(len(text_ids))]

att_masks = []
for ids in text_ids:
    masks = [int(id > 0) for id in ids]
    att_masks.append(masks)

100%|██████████| 10855/10855 [00:06<00:00, 1797.14it/s]


In [53]:
text_ids = torch.tensor(text_ids);
att_masks = torch.tensor(att_masks);

In [54]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

test_data = TensorDataset(text_ids, att_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [55]:
outputs = []
with torch.no_grad():
    model.eval()
    for k, (mb_x, mb_m) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        mb_x = mb_x.to(device)
        mb_m = mb_m.to(device)
        output = model(mb_x, attention_mask=mb_m)
        outputs.append(output[0].to('cpu'))

outputs = torch.cat(outputs)

_, predicted_values = torch.max(outputs, 1)
predicted_values = predicted_values.numpy()


100%|██████████| 340/340 [00:57<00:00,  5.89it/s]


In [56]:
predicted_labels = ["sexist" if p==1 else "not sexist" for p in predicted_values]

In [57]:
data["predict"] = predicted_labels

In [61]:
data[["text", "label", "predict"]].to_csv("../GeneratedTexts/task_b_generated_text.csv", index=False)

## Shapley

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
# data = train

In [ ]:
!pip install -q shap

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
ls ../Models

1a_fine-tuned-bert/


In [ ]:
import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig

out_dir = f'../Models/1a_fine-tuned-bert'
model = BertForSequenceClassification.from_pretrained(out_dir)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [ ]:
import transformers
import torch
import numpy as np
import scipy as sp

In [ ]:
texts = train['text'].values

In [ ]:
import shap
# define a prediction function
def f(texts):
  text_ids = [tokenizer.encode(text, max_length=100, padding='max_length', truncation=True) for text in texts]

  att_masks = []
  for ids in text_ids:
      masks = [int(id > 0) for id in ids]
      att_masks.append(masks)

  text_ids = torch.tensor(text_ids).to(device)
  att_masks = torch.tensor(att_masks).to(device)

  outputs = model(text_ids, attention_mask=att_masks)
  outputs = outputs[0].detach().cpu().numpy()
  scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
  
  val = sp.special.logit(scores[:,1]) # use one vs rest logit units
  return val

In [ ]:
for label in label_values:
  print(label)

1.1 threats of harm
1.2 incitement and encouragement of harm
2.1 descriptive attacks
2.2 aggressive and emotive attacks
2.3 dehumanising attacks & overt sexual objectification
3.1 casual use of gendered slurs, profanities, and insults
3.2 immutable gender differences and gender stereotypes
3.3 backhanded gendered compliments
3.4 condescending explanations or unwelcome advice
4.1 supporting mistreatment of individual women
4.2 supporting systemic discrimination against women as a group


In [ ]:
# ls ../Results/TaskC

In [ ]:
# import pickle

# def save_shap_values(filepath, obj):
#   with open(filepath, 'wb') as fin:
#     pickle.dump(obj, fin)

# def save_lexicons(filepath, obj):
#   with open(filepath, 'w', encoding="utf-8") as fin:
#     for w in obj:
#       fin.write(str(w)+"\n")

# def get_lexicons(shap_values):
#   feature_names = shap_values.abs.mean(0).feature_names
#   shapley_values = shap_values.abs.mean(0).values

#   candidates = [x for v, x in sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0]) if v > 0]
#   return candidates
    

# explainer = shap.Explainer(f, tokenizer)

# shap_values = {}
# lexicons = {}
# for idx, label in enumerate(label_values):
#   print("Working on", label)
#   d = train[train[target_column]==label]
#   print("#Rows", len(d))
#   d = {"text": d["text"].values}
  
#   shap_values[label] = explainer(d, fixed_context=1, batch_size=256)
#   save_shap_values(f"../Results/TaskB/aug_gptj_shapley_values_{idx+1}.pickle", shap_values[label])
#   # lexicons[label] = get_lexicons(shap_values[label])
#   # save_lexicons(f"./Results/shapley_lexicon_{idx+1}.txt", lexicons[label])
#   print("")

In [ ]:
ls ../Results/TaskB

aug_gptj_shapley_values_1.pickle        lexicon_shapley_augmented_v2_sqrt.json
aug_gptj_shapley_values_2.pickle        lexicon_shapley_augmented_v3_gptj.json
aug_gptj_shapley_values_3.pickle        lexicon_shapley.json
aug_gptj_shapley_values_4.pickle        shapley_values_1.pickle
lexicon_pmi.json                        shapley_values_2.pickle
lexicon_shapley_augmented_v1.json       shapley_values_3.pickle
lexicon_shapley_augmented_v2_shap.json  shapley_values_4.pickle


In [ ]:
import pickle

def load_shap_values(filepath):
  with open(filepath, 'rb') as fin:
    obj = pickle.load(fin)
  return obj

def get_lexicons(shap_values):
  feature_names = shap_values.abs.mean(0).feature_names
  shapley_values = shap_values.abs.mean(0).values

  sorted_values = sorted(zip(shapley_values, feature_names), key=lambda pair: -pair[0])
  features = [x for v, x in sorted_values]
  values = {x:v for v, x in sorted_values}
  return features, values
  
shap_values = {}
lexicons = {}
lexicons_score = {}
for idx, label in enumerate(label_values):
  shap_values[label] = load_shap_values(f"../Results/TaskC/aug_gptj_shapley_values_{idx+1}.pickle")
  k, v = get_lexicons(shap_values[label])
  lexicons[label] = k
  lexicons_score[label] = v

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
# print("Category", label_values[0])
# shap.plots.bar(shap_values[label_values[0]].abs.mean(0))

In [ ]:
# print("Category", label_values[1])
# shap.plots.bar(shap_values[label_values[1]].abs.mean(0))

In [ ]:
# print("Category", label_values[2])
# shap.plots.bar(shap_values[label_values[2]].abs.mean(0))

In [ ]:
# print("Category", label_values[3])
# shap.plots.bar(shap_values[label_values[3]].abs.mean(0))

In [ ]:
# lexicons["2. derogation"]

# Handle subwords

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from collections import defaultdict
from tqdm import tqdm

def map_subwords(words, subwords):
  sidx = 0
  widx = 0
  w = ""
  mapping = []
  while widx < len(words):
    w = w + words[widx]
    if sidx >= len(subwords):
      print(widx, w, sidx)
      print(words)
      print(subwords)

    idx = [sidx]
    s = subwords[sidx].replace("#", "")

    while len(w) < len(s):
      widx += 1
      w = w + words[widx].strip()

    # if words[0] == "layin":
    #     print(w, s)

    while s!=w and len(s) < len(w) and sidx+1 < len(subwords):
      sidx += 1
      s += subwords[sidx].replace("#", "").strip()
      idx.append(sidx)

      # if words[0] == "layin":
      #   print(f"[{w}] [{s}]", len(w), len(s), w==s)

    if len(s) > len(w):
      widx += 1
      sidx -= len(idx) - 1
      continue
      
    sidx += 1
    # print(w, s, idx)
    widx += 1
    w = ""

    mapping.append(idx)

  return mapping

In [ ]:
def clean_and_tokenize(sent):
  sent = sent.lower().replace("#", "").replace("''", '"')

  sent = sent.encode("ascii", "ignore")
  sent = sent.decode()
  # sent = sent.replace("``", '"')
  # sent = sent.replace("`", "'")
  
  words = nltk.word_tokenize(sent)
  for widx, w in enumerate(words):
    if w=="``":
      words[widx] = '"'
      continue
    elif w=="''":
      words[widx] = '"'
      continue
  
  # for w in words:
  #   vocabs.add(w)
    
  subwords = tokenizer.tokenize(sent)

  for sidx, s in enumerate(subwords):
    if s=="``":
      subwords[sidx] = '"'
      continue
    elif s=="''":
      subwords[sidx] = '"'
      continue
  return words, subwords


def merge_score(lexicons_score, new_lexicons, idxs, subwords):
  for idx in idxs:
    w = ""
    s = 0
    for i in idx:
      sw = subwords[i].replace("#", "")
      w += sw
      if sw not in lexicons_score:
        # print(">>", sw)
        continue

      s += lexicons_score[sw]
    
    new_lexicons[w] = s

In [ ]:
new_lexicons_scores = {}
new_lexicons_keys = {}

for idx, label in enumerate(label_values):
  print("Working on", label)
  new_lexicons = defaultdict(int)
  # vocabs[label] = set()

  d = train[train[target_column]==label]
  texts = d["text"].values
  for sent in tqdm(texts, total=len(texts)):
    try:
      words, subwords = clean_and_tokenize(sent)
      idxs = map_subwords(words, subwords)
      merge_score(lexicons_score[label], new_lexicons, idxs, subwords)
    except Exception as e:
      print("Error", str(e))
  
  sorted_list = sorted(new_lexicons.items(), key=lambda item: -item[1])
  new_lexicons_scores[label] = { k:v for k, v in sorted_list if v > 0}
  new_lexicons_keys[label] = [k for k, v in sorted_list if v > 0]

Working on 1.1 threats of harm


100%|██████████| 1294/1294 [00:01<00:00, 760.63it/s]


Working on 1.2 incitement and encouragement of harm


 77%|███████▋  | 1058/1374 [00:01<00:00, 632.27it/s]

11 ! 47
['*', 'chad', 'and', 'normie', 'are', 'literally', 'like', 'the', 'ferrrrrrrrrrrrrriest', 'couple', 'everrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!']
['*', 'chad', 'and', 'norm', '##ie', 'are', 'literally', 'like', 'the', 'fe', '##rr', '##rr', '##rr', '##rr', '##rr', '##rr', '##rries', '##t', 'couple', '[UNK]', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!']
Error list index out of range


...

100%|██████████| 1259/1259 [00:01<00:00, 789.98it/s]


Working on 4.1 supporting mistreatment of individual women


100%|██████████| 1314/1314 [00:01<00:00, 831.83it/s]


Working on 4.2 supporting systemic discrimination against women as a group


100%|██████████| 1249/1249 [00:01<00:00, 736.05it/s]


In [ ]:
# new_lexicons_scores

In [ ]:
ls ../Results/TaskC/

aug_gptj_shapley_values_10.pickle  lexicon_shapley.json
aug_gptj_shapley_values_11.pickle  shapley_values_10.pickle
aug_gptj_shapley_values_1.pickle   shapley_values_11.pickle
aug_gptj_shapley_values_2.pickle   shapley_values_1.pickle
aug_gptj_shapley_values_3.pickle   shapley_values_2.pickle
aug_gptj_shapley_values_4.pickle   shapley_values_3.pickle
aug_gptj_shapley_values_5.pickle   shapley_values_4.pickle
aug_gptj_shapley_values_6.pickle   shapley_values_5.pickle
aug_gptj_shapley_values_7.pickle   shapley_values_6.pickle
aug_gptj_shapley_values_8.pickle   shapley_values_7.pickle
aug_gptj_shapley_values_9.pickle   shapley_values_8.pickle
lexicon_pmi.json                   shapley_values_9.pickle


In [ ]:
import json

with open('../Results/TaskC/lexicon_shapley.json') as fin:
    lexicon_shapley = json.load(fin)

In [ ]:
test

,text,label,label_vector
5227,* I thought it was funny when I heard that. I ...,"3.1 casual use of gendered slurs, profanities,...","3.1 casual use of gendered slurs, profanities,..."
15190,"* I'm such a good girl, I'd do anything for an...",3.3 backhanded gendered compliments,3.3 backhanded gendered compliments
3944,"* I am worried for this baby, especially if he...",1.2 incitement and encouragement of harm,1.2 incitement and encouragement of harm
14672,* The only time I've been called a cunt was wh...,3.3 backhanded gendered compliments,3.3 backhanded gendered compliments
7721,* Some people do want to have a life free from...,4.2 supporting systemic discrimination against...,4.2 supporting systemic discrimination against...
...,...,...,...
16067,* We all know Normie's girls are being fed well.,4.1 supporting mistreatment of individual women,4.1 supporting mistreatment of individual women
16937,* This site is all I have. Every time I think ...,1.1 threats of harm,1.1 threats of harm
35,"* That's right, you're probably going to walk ...",2.3 dehumanising attacks & overt sexual object...,2.3 dehumanising attacks & overt sexual object...
473,* Why did you ask me for my phone number just ...,2.3 dehumanising attacks & overt sexual object...,2.3 dehumanising attacks & overt sexual object...


In [ ]:
test_texts = test["text"].values
test_words = [nltk.word_tokenize(t.lower()) for t in test_texts]
test_labels = test[target_column].values

In [ ]:
for l in lexicon_shapley:
  print(l)

1.1 threats of harm
1.2 incitement and encouragement of harm
2.1 descriptive attacks
2.2 aggressive and emotive attacks
2.3 dehumanising attacks & overt sexual objectification
3.1 casual use of gendered slurs, profanities, and insults
3.2 immutable gender differences and gender stereotypes
3.3 backhanded gendered compliments
3.4 condescending explanations or unwelcome advice
4.1 supporting mistreatment of individual women
4.2 supporting systemic discrimination against women as a group


In [ ]:
import numpy as np
threshold = {}
for l in new_lexicons_scores:
  values = [v for k, v in new_lexicons_scores[l].items()]
  threshold[l] = np.quantile(values, 0.3)

# print(threshold)

lexicon_values = {}
lexicon_keys = {}

for l in new_lexicons_scores:
  lexicon_values[l] = {k:v for k, v in lexicon_shapley[l].items()}
  lexicon_keys[l] = {k for k, v in lexicon_shapley[l].items()}
  for w in new_lexicons_scores[l]:
    v = new_lexicons_scores[l][w]
    if v > threshold[l]:
        if w in lexicon_keys[l]:
          lexicon_values[l][w] = max(v, lexicon_values[l][w])
        else:
          lexicon_keys[l].add(w)
          lexicon_values[l][w] = v
      
  print(l, len(lexicon_keys[l]))

1.1 threats of harm 1527
1.2 incitement and encouragement of harm 1831
2.1 descriptive attacks 2129
2.2 aggressive and emotive attacks 2049
2.3 dehumanising attacks & overt sexual objectification 1935
3.1 casual use of gendered slurs, profanities, and insults 2160
3.2 immutable gender differences and gender stereotypes 1966
3.3 backhanded gendered compliments 1525
3.4 condescending explanations or unwelcome advice 1562
4.1 supporting mistreatment of individual women 1591
4.2 supporting systemic discrimination against women as a group 1823


In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# %matplotlib inline

# plt.hist(values, bins=50)  # density=False would make counts
# plt.ylabel('Probability')
# plt.xlabel('Data');

In [ ]:
import sklearn
def predict(word, lexicon_values, lexicon_keys):
  cnt = {}
  for label in label_values:
    cnt[label] = 0

  for w in word:
    for label in label_values:
      if w in lexicon_keys[label]:
        cnt[label] += lexicon_values[label][w]

  
  if sum(cnt.values())==0:
    return None
  
  return max(cnt.items(), key=lambda k: k[1])[0]

y_pred = []
y_test = []
non = 0
for word, label in zip(test_words, test_labels):
  pred = predict(word, lexicon_values, lexicon_keys)
  if pred is None:
    non += 1
    continue

  y_pred.append(pred)
  y_test.append(label)

print("F1:", sklearn.metrics.f1_score(y_test, y_pred, average='macro'))
print("Skip:", non/len(test_words))

F1: 0.09869968954404751
Skip: 0.0


In [ ]:
# for i in range(1, 10):
#   print("Threshold", i/10)
#   threshold = {}
#   for l in new_lexicons_scores:
#     values = [v for k, v in new_lexicons_scores[l].items()]
#     threshold[l] = np.quantile(values, i/10)

#   # print(threshold)

#   lexicon_values = {}
#   lexicon_keys = {}
#   for l in new_lexicons_scores:
#     lexicon_values[l] = {k:v for k, v in lexicon_shapley[l].items()}
#     lexicon_keys[l] = {k for k, v in lexicon_shapley[l].items()}
#     for w in new_lexicons_scores[l]:
#       v = new_lexicons_scores[l][w]

#       if v > threshold[l]:

#         if w in lexicon_keys[l]:
#           lexicon_values[l][w] = max(v, lexicon_values[l][w])
#         else:
#           lexicon_keys[l].add(w)
#           lexicon_values[l][w] = v

#   y_pred = []
#   y_test = []
#   non = 0
#   for word, label in zip(test_words, test_labels):
#     pred = predict(word, lexicon_values, lexicon_keys)
#     if pred is None:
#       non += 1
#       continue

#     y_pred.append(pred)
#     y_test.append(label)

#   print("F1:", sklearn.metrics.f1_score(y_test, y_pred, average='macro'), non/len(test_words))

In [ ]:
import json
with open('../Results/TaskC/lexicon_shapley_augmented_v3_gptj.json', 'w') as outfile:
    json.dump(lexicon_values, outfile)